In [ ]:
import pandas as pd
import numpy as np

np.random.seed(1349)

In [ ]:
# Create list of values for names column.

students = ['Sally', 'Jane', 'Suzie', 'Billy', 'Ada', 'John', 'Thomas',
            'Marie', 'Albert', 'Richard', 'Isaac', 'Alan']

# Randomly generate arrays of scores for each student for each subject.
# Note that all the values need to have the same length here.

math_grades = np.random.randint(low=60, high=100, size=len(students))
english_grades = np.random.randint(low=60, high=100, size=len(students))
reading_grades = np.random.randint(low=60, high=100, size=len(students))

In [ ]:
df = pd.DataFrame({'name': students,
                   'math': math_grades,
                   'english': english_grades,
                   'reading': reading_grades,
                   'classroom': np.random.choice(['A', 'B'], len(students))})

## Indexing and Subsetting

Like the pandas Series object, the pandas DataFrame object supports both position- and label-based indexing using the indexing operator `[]`.

I will demonstrate concrete examples of indexing using the indexing operator `[]` alone and with the `.loc` and `.iloc` attributes below.

### `[]`

I can pass a list of columns from a DataFrame to the indexing operator (aka bracket notation) to return a subset of my original DataFrame.

In [ ]:
df.name

In [ ]:
# I can choose a single column using bracket notation.



In [ ]:
# Choose only two columns for my subset.



In [ ]:
# I can pass a boolean Series to the indexing operator as a selector.
# names that start with a?


In [ ]:
df[bools]

### `.loc`

We can use the `.loc` attribute to select specific rows AND columns by index label. The index label can be a number, but it can also be a string label. This method offers a lot of flexibility! **The .loc attribute's indexing is inclusive and uses an index label, not position.**

```python
df.loc[row_indexer, column_indexer]
```

In [ ]:
# Select all the rows and a subset of columns; notice the inclusive behavior of the indexing.



In [ ]:
# I can use a boolean Series as a selector with .loc, too, but I can choose rows and columns.



### `.iloc`

We can use the `.iloc` attribute to select specific rows and colums by index position. `.iloc` does not accept a boolean Series as a selector like `.loc` does. **It takes in integers representing index position and is NOT inclusive.**

```python
df.iloc[row_indexer, column_indexer]
```

We can select rows by integer position:

In [ ]:
# Notice the exclusive behavior of the indexing.

df.iloc[:3]

We can also specify which columns we want to select:

In [ ]:
df.iloc[:3, 1:4]

Here we select the first 3 rows (everything up to but not including the index of 3), and the second and third columns (starting from the index of 1 up to but not including the index of 3).

## Aggregating

### `.agg`

The `.agg` method lets us specify a way to aggregate a series of numerical values. We pass an aggregate function or list of functions to the method that we want applied to a Series.

In [ ]:
# I can use it on the entire df.



In [ ]:
df.describe()

In [ ]:
# I can use it on a single column in a df.



In [ ]:
# I can pass a list of functions to the .agg method.



While on the surface this seems pretty simple, `.agg` is capable of providing more detailed aggregations:

### `.groupby`

The `.groupby` method is used to create a grouped object, which we can then apply an aggregation on. For example, if we wanted to know the highest math grade from each classroom:

In [ ]:
df.groupby('classroom').math.max()

We can use `.agg` here to, to see multiple aggregations:

In [ ]:
groupby1 = df.groupby('classroom').math.agg(['min', 'mean', 'max'])
groupby1.columns = ['math_min_grade', 'math_avg_grade', 'math_max_grade']
groupby1

We can group by multiple columns as well. To demonstrate, we'll create a boolean column named `passing_math`, then group by the combination of our new feature, `passing_math`, and the classroom and calculate the average reading grade and the number of individuals in each subgroup. 

Let's break this problem down and code it step-by-step.

#### `np.where`

First, we can create the new `passing_math` column using a handy NumPy function called `np.where`. It will allow us to base the new column values on whether the values in an existing column, `math`, meet a condition.

```python
np.where(condition, this_where_True, this_where_False)
```

In [ ]:
# Create the new column based on an existing column.

df['passing_math'] = np.where(df.math < 70, 'failing', 'passing')
df

Next, we will group by the `passing_math` and `classroom` columns and use the `.agg` method to calculate the average reading grade and the number of students.

In [ ]:
grade_groups = df.groupby(['passing_math', 'classroom']).reading.agg(['mean', 'count'])
grade_groups

In [ ]:
# I can even clean up my columns to make my calculations clearer.

grade_groups.columns = ['avg_reading_grade', 'count_of_students']
grade_groups

**Takeaways:**

We can interpret this output as there being 2 students failing math in classroom A with an average reading grade of 87, 6 students passing math in classroom A with an average reading grade of 87.16, and 4 students passing math in classroom B with an average reading grade of 85.25.

#### `.transform`

The `.transform` method can be used to produce a series with the same length of the original dataframe where each value represents the aggregation from the subgroup resulting from the `.groupby`. 

For example, if we wanted to know the average math grade for each classroom and add this data back to our original dataframe:

In [ ]:
df.assign(avg_math_score_by_classroom=df.groupby('classroom').math.transform('mean'))

#### `.describe`

Check out what I can do when I combine a `.groupby` with a `.describe`!

In [ ]:
df.groupby('classroom').reading.describe()

## Merging and Joining

Pandas provides several ways to combine dataframes together. We will look at two of them below:

### `pd.concat`

This function takes in a list or dictionary of Series or DataFrame objects and joins them along a particular axis, row-wise axis=0 or column-wise axis=1.

```python
# For example, concat with a list of two DataFrames
pd.concat([df1, df2], axis=0)
```

- When your list contains at least one DataFrame, a DataFrame is returned.


- When concatenating only Series objects row-wise, axis=0, a Series is returned.


- When concatenating Series or DataFrames column-wise, axis=1, a DataFrame is returned.

```python
# Default is set to row-wise concatenation using an outer join.
pd.concat(objs, axis=0, join='outer')
```

When concatenating dataframes vertically, we basically are just adding more rows to an existing dataframe. In this case, the dataframes we are putting together should have the same column names[^1].

In [ ]:
df1 = pd.DataFrame({'a': [1, 2, 3]})
df2 = pd.DataFrame({'a': [4, 5, 6]})

df1

In [ ]:
df2

In [ ]:
pd.concat(

**Note** that the indices are preserved on the resulting dataframe; we could set the `ignore_index` parameter to `True` if we wanted these to be sequential.

In [ ]:
concat_df1 = pd.concat([df1, df2], ignore_index=True)
concat_df1

[^1]:
    We can concatenate dataframes with different column names, but generally this is not the behavior we want, as pandas will fill in a lot of null values into the resulting dataframe. The exception to this is if the dataframes are aligned on their index (i.e. the labels for each row), then we can provide the `axis=1` keyword argument to `pd.concat` to merge the dataframes horizontally.

In [ ]:
concat_df2 = pd.DataFrame({'b': [1, 2, 3, 4, 5, 6]})
concat_df2

In [ ]:
pd.concat([concat_df1, concat_df2], axis=1)

In [ ]:
pd.concat([concat_df1, df1], axis=1)

### `.merge`

This method is similar to a SQL join. Here's a [cool read](https://pandas.pydata.org/pandas-docs/stable/getting_started/comparison/comparison_with_sql.html#compare-with-sql-join) making a comparison between the two, if you're interested.

```python
# df.merge default settings for commonly used parameters.

left_df.merge(right_df, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, indicator=False)
```

How does changing the default argument of the `how` parameter change my resulting DataFrame?

`how` == Type of merge to be performed.

`how=left`: use only keys from left frame, similar to a SQL left outer join; preserve key order.

`how=right`: use only keys from right frame, similar to a SQL right outer join; preserve key order.

`how=outer`: use union of keys from both frames, similar to a SQL full outer join; sort keys lexicographically.

`how=inner`: use intersection of keys from both frames, similar to a SQL inner join; preserve the order of the left keys.

In [ ]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users

In [ ]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles

The `.merge` method will allow us to specify `left_on` and `right_on` to indicate the columns that are the keys used to merge the dataframes together. 

- In addition, the `how` keyword argument is used to define what type of JOIN we want to do; as we saw above, `inner` is the default setting. 

- For demonstration purposes, I'm also going to set the `indicator` parameter to `True`, which will create a column indicating whether the merge key appears in the `left_only`, `right_only` or `both` DataFrames.

In [ ]:
# Perform an outer join specifying the left and right DataFrame keys.

users.merge(roles, left_on='role_id', right_on='id', how='outer', indicator=True)

Notice that we have duplicate column names in the resulting dataframe. By default, pandas will add a suffix of `_x` to any columns in the left dataframe that are duplicated, and `_y` to any columns in the right dataframe that are duplicated. I can clean up my columns if I want to; one way would be to use method chaining, which it demonstrated below:

In [ ]:
temp = (users.merge(roles, 
            left_on='role_id', 
            right_on='id', 
            how='outer')
    .drop(columns='role_id')
    .rename(columns={'id_x': 'id', 
                     'name_x': 'employee',
                     'id_y': 'role_id',
                     'name_y': 'role'}
            )
)
temp

In [ ]:
temp.info()